In [14]:
import pandas as pd
import numpy as np
import re

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer


volunteer_org = pd.read_csv("./data/volunteer_opportunities.csv")
wine = pd.read_csv("./data/wine_types.csv")

X = wine[["Proline", "Total phenols", "Hue", "Nonflavanoid phenols"]]
y = wine["Type"]

hiking = pd.read_json("./data/hiking.json")

running_times_5k = pd.DataFrame({
    "name": ["Sue", "Mark", "Sean", "Erin", "Jenny", "Russell"],
    "run1": [20.1, 16.5, 23.5, 21.7, 25.8, 30.9],
    "run2": [18.5, 17.1, 25.1, 21.1, 27.1, 29.6],
    "run3": [19.6, 16.9, 25.2, 20.9, 26.1, 31.4],
    "run4": [20.3, 17.6, 24.6, 22.1, 26.7, 30.4],
    "run5": [18.3, 17.3, 23.9, 22.2, 26.9, 29.9]})

volunteer = volunteer_org[['vol_requests', 'title', 'hits', 'category_desc', 'locality', 'region', 'postalcode', 'created_date']]

In [15]:
# Create a list of redundant column names to drop
to_drop = ["category_desc", "created_date", "locality", "region", "vol_requests"]

# Drop those columns from the dataset
volunteer_subset = volunteer.drop(to_drop, axis=1)

# Print out the head of volunteer_subset
print(volunteer_subset.head())

                                               title  hits  postalcode
0  Volunteers Needed For Rise Up & Stay Put! Home...   737         NaN
1                                       Web designer    22     10010.0
2      Urban Adventures - Ice Skating at Lasker Rink    62     10026.0
3  Fight global hunger and support women farmers ...    14      2114.0
4                                      Stop 'N' Swap    31     10455.0


In [16]:
# Print out the column correlations of the wine dataset
print(wine.corr())

# Drop that column from the DataFrame
wine = wine.drop(["Flavanoids"], axis=1)

print(wine.head())

                                  Type   Alcohol  Malic acid       Ash  \
Type                          1.000000 -0.328222    0.437776 -0.049643   
Alcohol                      -0.328222  1.000000    0.094397  0.211545   
Malic acid                    0.437776  0.094397    1.000000  0.164045   
Ash                          -0.049643  0.211545    0.164045  1.000000   
Alcalinity of ash             0.517859 -0.310235    0.288500  0.443367   
Magnesium                    -0.209179  0.270798   -0.054575  0.286587   
Total phenols                -0.719163  0.289101   -0.335167  0.128980   
Flavanoids                   -0.847498  0.236815   -0.411007  0.115077   
Nonflavanoid phenols          0.489109 -0.155929    0.292977  0.186230   
Proanthocyanins              -0.499130  0.136698   -0.220746  0.009652   
Color intensity               0.265668  0.546364    0.248985  0.258887   
Hue                          -0.617369 -0.071747   -0.561296 -0.074667   
OD280/OD315 of diluted wines -0.788230

In [17]:
## I have no idea what the vocab, tfidf_vec.vocabulary and text_tfidf are

'''
# Add in the rest of the arguments
def return_weights(vocab, original_vocab, vector, vector_index, top_n):
    zipped = dict(zip(vector[vector_index].indices, vector[vector_index].data))
    
    # Transform that zipped dict into a series
    zipped_series = pd.Series({vocab[i]:zipped[i] for i in vector[vector_index].indices})
    
    # Sort the series to pull out the top n weighted words
    zipped_index = zipped_series.sort_values(ascending=False)[:top_n].index
    return [original_vocab[i] for i in zipped_index]

# Print out the weighted words
print(return_weights(vocab, tfidf_vec.vocabulary_, text_tfidf, 8, 3))
'''

'\n# Add in the rest of the arguments\ndef return_weights(vocab, original_vocab, vector, vector_index, top_n):\n    zipped = dict(zip(vector[vector_index].indices, vector[vector_index].data))\n    \n    # Transform that zipped dict into a series\n    zipped_series = pd.Series({vocab[i]:zipped[i] for i in vector[vector_index].indices})\n    \n    # Sort the series to pull out the top n weighted words\n    zipped_index = zipped_series.sort_values(ascending=False)[:top_n].index\n    return [original_vocab[i] for i in zipped_index]\n\n# Print out the weighted words\nprint(return_weights(vocab, tfidf_vec.vocabulary_, text_tfidf, 8, 3))\n'

In [18]:
## Same as before

'''
def words_to_filter(vocab, original_vocab, vector, top_n):
    filter_list = []
    for i in range(0, vector.shape[0]):
    
        # Call the return_weights function and extend filter_list
        filtered = return_weights(vocab, original_vocab, vector, i, top_n)
        filter_list.extend(filtered)
        
    # Return the list in a set, so we don't get duplicate word indices
    return set(filter_list)

# Call the function to get the list of word indices
filtered_words = words_to_filter(vocab, tfidf_vec.vocabulary_, text_tfidf, 3)

# Filter the columns in text_tfidf to only those in filtered_words
filtered_text = text_tfidf[:, list(filtered_words)]
'''

"\ndef words_to_filter(vocab, original_vocab, vector, top_n):\n    filter_list = []\n    for i in range(0, vector.shape[0]):\n    \n        # Call the return_weights function and extend filter_list\n        filtered = return_weights(vocab, original_vocab, vector, i, top_n)\n        filter_list.extend(filtered)\n        \n    # Return the list in a set, so we don't get duplicate word indices\n    return set(filter_list)\n\n# Call the function to get the list of word indices\nfiltered_words = words_to_filter(vocab, tfidf_vec.vocabulary_, text_tfidf, 3)\n\n# Filter the columns in text_tfidf to only those in filtered_words\nfiltered_text = text_tfidf[:, list(filtered_words)]\n"

In [19]:
## Same

'''
# Split the dataset according to the class distribution of category_desc
X_train, X_test, y_train, y_test = train_test_split(filtered_text.toarray(), y, stratify=y, random_state=42)

# Fit the model to the training data
nb.fit(X_train, y_train)

# Print out the model's accuracy
print(nb.score(X_test, y_test))
'''

"\n# Split the dataset according to the class distribution of category_desc\nX_train, X_test, y_train, y_test = train_test_split(filtered_text.toarray(), y, stratify=y, random_state=42)\n\n# Fit the model to the training data\nnb.fit(X_train, y_train)\n\n# Print out the model's accuracy\nprint(nb.score(X_test, y_test))\n"

In [20]:
# Instantiate a PCA object
pca = PCA()

# Define the features and labels from the wine dataset
X = wine.drop("Type", axis=1)
y = wine["Type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Apply PCA to the wine dataset X vector
pca_X_train = pca.fit_transform(X_train)
pca_X_test = pca.transform(X_test)

# Look at the percentage of variance explained by the different components
print(pca.explained_variance_ratio_)

[9.97802349e-01 2.02071713e-03 9.82348559e-05 5.53994004e-05
 1.10395648e-05 5.87233448e-06 3.13858204e-06 1.54420449e-06
 1.02927386e-06 3.90521513e-07 1.95535151e-07 8.99659634e-08]


In [21]:
knn = KNeighborsClassifier()

In [22]:
# Fit knn to the training data
knn.fit(pca_X_train, y_train)

# Score knn on the test data and print it out
print(knn.score(pca_X_test, y_test))

0.7777777777777778
